In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ultralytics import YOLO
from PIL import Image
import requests
from io import BytesIO
import cv2
import math
print("Hello Word")

Hello Word


In [24]:
model = YOLO("best.pt")

In [39]:
vid = cv2.VideoCapture('DoorOconnel1.mp4')

if (vid.isOpened()== False):
    print("Error opening video")

while vid.isOpened():
    ret, frame = vid.read()
    blur = cv2.GaussianBlur(frame,(11,11),0)
    if not ret:
      break

    else:
      result = model.predict(blur)
      if len(result[0].boxes)==0:
         cv2.imshow("test", result[0].orig_img)
         continue
      cordinates = result[0].boxes[0].xyxy.tolist()
      start = (int(cordinates[0][0]), int(cordinates[0][1]))
      end = (int(cordinates[0][2]), int(cordinates[0][3]))
      color = (255, 0, 0)
      thickness=2

      image = cv2.rectangle(frame, start, end, color, thickness)
      cv2.imshow("test", image)
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()


0: 640x352 1 1, 1 2, 105.1ms
Speed: 3.0ms preprocess, 105.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 115.5ms
Speed: 2.0ms preprocess, 115.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 109.4ms
Speed: 1.0ms preprocess, 109.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 96.0ms
Speed: 1.0ms preprocess, 96.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 103.6ms
Speed: 2.0ms preprocess, 103.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 133.9ms
Speed: 1.2ms preprocess, 133.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 104.7ms
Speed: 0.5ms preprocess, 104.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x352 1 1, 1 2, 102.0ms
Speed: 2.0ms preprocess, 102.0ms inference, 0.0ms postprocess per image at s

In [35]:
def ratioPixToInch(pixH, incH, pixW, incW):
    ratio1 = pixH/incH
    ratio2 = pixW/incW
    ratio = (ratio1+ratio2)/2
    return ratio

def pixToInch(ratio, pix):
    inc = pix/ratio
    return inc

def getFocal(dis, wid, widImg, hei, heiImg):
    focal1 = (widImg * dis)/ wid
    focal2 = (heiImg * dis)/ hei
    focal = (focal1+focal2)/2
    return focal1

def getDistance(focal, wid, widImg):
    distance = (wid * focal)/widImg
    return distance

In [40]:
def trainDoorDistance(realDoorWidth, realDoorHeight, distanceToDoor, resultList=result[0].boxes[0].xyxy.tolist()):
    
    widthImgDoor = abs(int(resultList[0][0]) - int(resultList[0][2]))
    heightImgDoor = abs(int(resultList[0][1]) - int(resultList[0][3]))

    ratio = ratioPixToInch(widthImgDoor, realDoorWidth, heightImgDoor, realDoorHeight)

    inchImgDoorW = pixToInch(ratio, widthImgDoor)
    inchImgDoorHeight = pixToInch(ratio, heightImgDoor)

    focal = getFocal(distanceToDoor, realDoorWidth, inchImgDoorW, realDoorHeight, inchImgDoorHeight)
    # print(focal)
    return focal, inchImgDoorW, ratio, realDoorWidth

In [41]:
focalPoint, widthInImage, ratio, doorWidth = trainDoorDistance(42, 80, 120)

In [42]:
print(focalPoint)
print(widthInImage)
print(ratio)
print(doorWidth)

122.40911557243624
42.84319045035269
5.4851190476190474
42
